# Eksamen Datadrevet Organisationsanalyse efterår 2021

Peter Witfelt Larsen, Frederik Valeur & Marcus Kunøe

In [4]:
# Loading packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns

# 1.0 Import og inspektion af data

## 1.1 Oversigtsarket 

In [5]:
# Impoterer xlsx-filer 

df = pd.read_excel (r"C:\Users\Marcu\OneDrive\Skrivebord\DO2021\XLSX\Oversigt over KK materiel(1).xlsx")

In [6]:
df.head()

,Sorteringsfelt,Køretøjs-ID-nr.,Producent,Typebetegnelse,Hovedkategorinavn,Underkategori,Hovedkategorinavn.1,Prim. brændstof,Ejerskab,Anskaf.værdi,...,Enhed,Levetid i år (afskrivning),Anhængertræk,Hejs,Kran,Lift,Tip,Præmie Ansvar,Præmie Kasko,Forsikring
0,542494,24-94,MASSEY-FERGUSON,2430-4WD TRAKTOR,Traktorer,LANDBRUG,Traktorer,DIESEL,Ejet,212000.0,...,PKR - Grøn Drift Syd,8.0,Ingen,Nej,Nej,Nej,Nej,3200,1100,Ansvar & Kasko
1,542530,25-30,LM TRAC,TYPE 385,"Minitraktorer, Redskabsbærere",REDSKABSBÆRER,"Minitraktorer, Redskabsbærere",DIESEL,Ejet,426667.0,...,PKR - Istandsættelse,8.0,Synsfrit,Nej,Nej,Nej,Nej,3200,1302,Ansvar & Kasko
2,542656,26-56,SCANIA,P230 DB 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,866900.0,...,MKB - Vejvedligehold,8.0,Ingen,Nej,Nej,Nej,Nej,3200,1928,Ansvar & Kasko
3,542666,26-66,MAN,12.210 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,860721.0,...,MKB - Vejvedligehold,8.0,Synsfrit,Ja,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200,2127,Ansvar & Kasko
4,542675,26-75,MAN,15.240 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,910928.0,...,MKB - Vejvedligehold,8.0,Synsfrit,Nej,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200,2779,Ansvar & Kasko


In [7]:
print(df.loc[range(100),['Sorteringsfelt', 'Underkategori', 'Enhed']])

    Sorteringsfelt      Underkategori                          Enhed
0           542494           LANDBRUG           PKR - Grøn Drift Syd
1           542530      REDSKABSBÆRER           PKR - Istandsættelse
2           542656  LASTVOGN 2-AKSLET           MKB - Vejvedligehold
3           542666  LASTVOGN 2-AKSLET           MKB - Vejvedligehold
4           542675  LASTVOGN 2-AKSLET           MKB - Vejvedligehold
..             ...                ...                            ...
95         5426666  LASTVOGN 3-AKSLET         PKR - Kørsel og Vinter
96         5426700  LASTVOGN 2-AKSLET     PKR - Byinventar og Cykler
97         5427027            MINIBUS  STAB - Organisation & Ledelse
98         5427127            MINIBUS   PARC - Jord og Genanvendelse
99         5427221            MINIBUS   PARC - Jord og Genanvendelse

[100 rows x 3 columns]


In [8]:
df.rename(columns={'Sorteringsfelt': 'ID#'}, inplace=True)

## 1.2 GPS data

In [9]:
df2 = pd.read_excel (r"C:\Users\Marcu\OneDrive\Skrivebord\DO2021\XLSX\GPS data 2020(1).xlsx")

In [10]:
df2.head()

,nummer,forvaltning,placeringkode,dato,måned,km kørsel,timer tænding
0,5524704,ØKF,5678.0,2020-08-25,august,1.810,0.696111
1,5524704,ØKF,5678.0,2020-08-26,august,4.234,1.271944
2,5524704,ØKF,5678.0,2020-08-27,august,1.760,1.029722
3,5524704,ØKF,5678.0,2020-08-28,august,1.785,0.683611
4,5524704,ØKF,5678.0,2020-08-31,august,1.806,0.797778


In [11]:
df2.rename(columns={'nummer': 'ID#'}, inplace=True)
df2.head()

,ID#,forvaltning,placeringkode,dato,måned,km kørsel,timer tænding
0,5524704,ØKF,5678.0,2020-08-25,august,1.810,0.696111
1,5524704,ØKF,5678.0,2020-08-26,august,4.234,1.271944
2,5524704,ØKF,5678.0,2020-08-27,august,1.760,1.029722
3,5524704,ØKF,5678.0,2020-08-28,august,1.785,0.683611
4,5524704,ØKF,5678.0,2020-08-31,august,1.806,0.797778


# Datawrangling

## Sorterer GPS data og summerer flere variable

In [12]:
df2["Count"] = 1
df3 = df2.groupby("ID#").aggregate({"placeringkode": "min", "km kørsel": "sum", "timer tænding": "sum", "Count": "sum"})

print(df3)

         placeringkode  km kørsel  timer tænding  Count
ID#                                                    
542492          1489.0      2.044       1.596111      4
542494          1531.0    166.303      48.129167     29
542523          1540.0     89.645      48.873333     29
542528          1541.0    348.037      62.479722     36
542530          4185.0    146.291     148.328611     60
...                ...        ...            ...    ...
5567773         5993.0     96.598     152.161667     52
5567774         5993.0      9.028      12.592778      8
5567775         5993.0     35.742      16.607500     20
5570436         3867.0      7.749       0.847778      3
5570905         3867.0    317.211      39.831389     70

[1387 rows x 4 columns]


In [13]:
df4 = pd.merge(df, df3, on='ID#', how='outer')

In [14]:
df4.head()

,ID#,Køretøjs-ID-nr.,Producent,Typebetegnelse,Hovedkategorinavn,Underkategori,Hovedkategorinavn.1,Prim. brændstof,Ejerskab,Anskaf.værdi,...,Kran,Lift,Tip,Præmie Ansvar,Præmie Kasko,Forsikring,placeringkode,km kørsel,timer tænding,Count
0,542494,24-94,MASSEY-FERGUSON,2430-4WD TRAKTOR,Traktorer,LANDBRUG,Traktorer,DIESEL,Ejet,212000.0,...,Nej,Nej,Nej,3200.0,1100.0,Ansvar & Kasko,1531.0,166.303,48.129167,29.0
1,542530,25-30,LM TRAC,TYPE 385,"Minitraktorer, Redskabsbærere",REDSKABSBÆRER,"Minitraktorer, Redskabsbærere",DIESEL,Ejet,426667.0,...,Nej,Nej,Nej,3200.0,1302.0,Ansvar & Kasko,4185.0,146.291,148.328611,60.0
2,542656,26-56,SCANIA,P230 DB 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,866900.0,...,Nej,Nej,Nej,3200.0,1928.0,Ansvar & Kasko,1489.0,3238.944,245.536389,152.0
3,542666,26-66,MAN,12.210 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,860721.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2127.0,Ansvar & Kasko,1487.0,9059.073,1119.422500,181.0
4,542675,26-75,MAN,15.240 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,910928.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2779.0,Ansvar & Kasko,1487.0,7445.753,730.576389,149.0


In [15]:
df4

,ID#,Køretøjs-ID-nr.,Producent,Typebetegnelse,Hovedkategorinavn,Underkategori,Hovedkategorinavn.1,Prim. brændstof,Ejerskab,Anskaf.værdi,...,Kran,Lift,Tip,Præmie Ansvar,Præmie Kasko,Forsikring,placeringkode,km kørsel,timer tænding,Count
0,542494,24-94,MASSEY-FERGUSON,2430-4WD TRAKTOR,Traktorer,LANDBRUG,Traktorer,DIESEL,Ejet,212000.0,...,Nej,Nej,Nej,3200.0,1100.0,Ansvar & Kasko,1531.0,166.303,48.129167,29.0
1,542530,25-30,LM TRAC,TYPE 385,"Minitraktorer, Redskabsbærere",REDSKABSBÆRER,"Minitraktorer, Redskabsbærere",DIESEL,Ejet,426667.0,...,Nej,Nej,Nej,3200.0,1302.0,Ansvar & Kasko,4185.0,146.291,148.328611,60.0
2,542656,26-56,SCANIA,P230 DB 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,866900.0,...,Nej,Nej,Nej,3200.0,1928.0,Ansvar & Kasko,1489.0,3238.944,245.536389,152.0
3,542666,26-66,MAN,12.210 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,860721.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2127.0,Ansvar & Kasko,1487.0,9059.073,1119.422500,181.0
4,542675,26-75,MAN,15.240 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,910928.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2779.0,Ansvar & Kasko,1487.0,7445.753,730.576389,149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,5567761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2296.0,2.581,0.983611,2.0
1592,5567769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5692.0,23.962,9.092778,11.0
1593,5567770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5692.0,25.979,11.214167,11.0
1594,5570436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3867.0,7.749,0.847778,3.0


In [16]:
df4.tail(300)

,ID#,Køretøjs-ID-nr.,Producent,Typebetegnelse,Hovedkategorinavn,Underkategori,Hovedkategorinavn.1,Prim. brændstof,Ejerskab,Anskaf.værdi,...,Kran,Lift,Tip,Præmie Ansvar,Præmie Kasko,Forsikring,placeringkode,km kørsel,timer tænding,Count
1296,5567806,67-806,HUSQVARNA,R420 ADW,Græsklipper,NaN,Græsklipper,DIESEL,Ejet,69104.0,...,Nej,Nej,Nej,3200.0,0.0,Ansvar & Kasko,NaN,NaN,NaN,NaN
1297,542492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1489.0,2.044,1.596111,4.0
1298,542523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1540.0,89.645,48.873333,29.0
1299,542528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1541.0,348.037,62.479722,36.0
1300,542533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1541.0,611.307,98.263056,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,5567761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2296.0,2.581,0.983611,2.0
1592,5567769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5692.0,23.962,9.092778,11.0
1593,5567770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5692.0,25.979,11.214167,11.0
1594,5570436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3867.0,7.749,0.847778,3.0


### Noterer, at der for observationerne >1296 kun er GPS data

In [17]:
pip install dplython

Note: you may need to restart the kernel to use updated packages.


In [18]:
df4


,ID#,Køretøjs-ID-nr.,Producent,Typebetegnelse,Hovedkategorinavn,Underkategori,Hovedkategorinavn.1,Prim. brændstof,Ejerskab,Anskaf.værdi,...,Kran,Lift,Tip,Præmie Ansvar,Præmie Kasko,Forsikring,placeringkode,km kørsel,timer tænding,Count
0,542494,24-94,MASSEY-FERGUSON,2430-4WD TRAKTOR,Traktorer,LANDBRUG,Traktorer,DIESEL,Ejet,212000.0,...,Nej,Nej,Nej,3200.0,1100.0,Ansvar & Kasko,1531.0,166.303,48.129167,29.0
1,542530,25-30,LM TRAC,TYPE 385,"Minitraktorer, Redskabsbærere",REDSKABSBÆRER,"Minitraktorer, Redskabsbærere",DIESEL,Ejet,426667.0,...,Nej,Nej,Nej,3200.0,1302.0,Ansvar & Kasko,4185.0,146.291,148.328611,60.0
2,542656,26-56,SCANIA,P230 DB 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,866900.0,...,Nej,Nej,Nej,3200.0,1928.0,Ansvar & Kasko,1489.0,3238.944,245.536389,152.0
3,542666,26-66,MAN,12.210 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,860721.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2127.0,Ansvar & Kasko,1487.0,9059.073,1119.422500,181.0
4,542675,26-75,MAN,15.240 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,910928.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2779.0,Ansvar & Kasko,1487.0,7445.753,730.576389,149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,5567761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2296.0,2.581,0.983611,2.0
1592,5567769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5692.0,23.962,9.092778,11.0
1593,5567770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5692.0,25.979,11.214167,11.0
1594,5570436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3867.0,7.749,0.847778,3.0


Inspicerer antallet af kolonner med missing values

In [19]:
df4.isnull()

,ID#,Køretøjs-ID-nr.,Producent,Typebetegnelse,Hovedkategorinavn,Underkategori,Hovedkategorinavn.1,Prim. brændstof,Ejerskab,Anskaf.værdi,...,Kran,Lift,Tip,Præmie Ansvar,Præmie Kasko,Forsikring,placeringkode,km kørsel,timer tænding,Count
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,False,False
1592,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,False,False
1593,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,False,False
1594,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,False,False


In [20]:
df4.isnull().any(axis = 0).sum()

33

Inspicerer antallet af rækker med missing values

In [21]:
df4.isnull().any(axis = 1).sum()

1039

Ovenstående kan vel ikke være korrekt, da den dropper væsentligt færre rækker nedenunder. Måske det er antallet af NaN i rækkerne? Hmm, nej... 

Vi dropper, hvor der er mindst en NaN værdi (Null)

In [22]:
df4.dropna(axis = 0)

,ID#,Køretøjs-ID-nr.,Producent,Typebetegnelse,Hovedkategorinavn,Underkategori,Hovedkategorinavn.1,Prim. brændstof,Ejerskab,Anskaf.værdi,...,Kran,Lift,Tip,Præmie Ansvar,Præmie Kasko,Forsikring,placeringkode,km kørsel,timer tænding,Count
0,542494,24-94,MASSEY-FERGUSON,2430-4WD TRAKTOR,Traktorer,LANDBRUG,Traktorer,DIESEL,Ejet,212000.0,...,Nej,Nej,Nej,3200.0,1100.0,Ansvar & Kasko,1531.0,166.303,48.129167,29.0
1,542530,25-30,LM TRAC,TYPE 385,"Minitraktorer, Redskabsbærere",REDSKABSBÆRER,"Minitraktorer, Redskabsbærere",DIESEL,Ejet,426667.0,...,Nej,Nej,Nej,3200.0,1302.0,Ansvar & Kasko,4185.0,146.291,148.328611,60.0
2,542656,26-56,SCANIA,P230 DB 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,866900.0,...,Nej,Nej,Nej,3200.0,1928.0,Ansvar & Kasko,1489.0,3238.944,245.536389,152.0
3,542666,26-66,MAN,12.210 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,860721.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2127.0,Ansvar & Kasko,1487.0,9059.073,1119.422500,181.0
4,542675,26-75,MAN,15.240 2-AKSLET,Lastvogne > 3500 kg,LASTVOGN 2-AKSLET,Lastvogne > 3500 kg,DIESEL,Ejet,910928.0,...,HIAB XS077 ES-3 HIDUO,Nej,Ja,3200.0,2779.0,Ansvar & Kasko,1487.0,7445.753,730.576389,149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,5567771,67-771,HUSQVARNA,R213C,Græsklipper,GRÆSKLIPPER 70-100CM,Græsklipper,DIESEL,Ejet,25000.0,...,Nej,Nej,Nej,3200.0,0.0,Ansvar & Kasko,5993.0,32.070,12.102778,19.0
1288,5567772,67-772,STIGA,COMBI 95,Græsklipper,GRÆSKLIPPER - UOPLYST,Græsklipper,DIESEL,Ejet,25000.0,...,Nej,Nej,Nej,3200.0,0.0,Ansvar & Kasko,5993.0,4.654,7.133056,10.0
1289,5567773,67-773,STIGA,100 COMBI,Græsklipper,GRÆSKLIPPER - UOPLYST,Græsklipper,DIESEL,Ejet,40000.0,...,Nej,Nej,Nej,3200.0,0.0,Ansvar & Kasko,5993.0,96.598,152.161667,52.0
1290,5567774,67-774,HUSQVARNA,R422TS,Græsklipper,GRÆSKLIPPER 101-150CM,Græsklipper,DIESEL,Ejet,95990.0,...,Nej,Nej,Nej,3200.0,0.0,Ansvar & Kasko,5993.0,9.028,12.592778,8.0


# I stedet for at droppe, ville det nok give mere mening at merge på en af de smarte måder, som Niklas har vist os. Og så sammenligne antallet af observationer, der droppes. 